In [1]:
import torch
from custom_models import load_custom_class
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# link = "cross-encoder/nli-distilroberta-base"
link = "cross-encoder/nli-MiniLM2-L6-H768"
tokenizer_nli, model_nli = load_custom_class(link, device=device)

In [2]:
import json
# data_path = "data/multinli_1.0/multinli_1.0_dev_mismatched.jsonl"
# data_path = "data/multinli_1.0/multinli_1.0_dev_matched.jsonl"
data_path = "data/snli_1.0/snli_1.0_test.jsonl"
data = {
        "label": [],
        "sent1": [],
        "sent2": [],
        "sent_combined": []
    }
with open(data_path, "r") as file:
    for line in file:
        aux_dict = json.loads(line)
        data["label"] += [aux_dict["gold_label"]]
        data["sent1"] += [aux_dict["sentence1"]] # premise
        data["sent2"] += [aux_dict["sentence2"]] # hypothesis
        data["sent_combined"] += [aux_dict["sentence1"] + " " + aux_dict["sentence2"]]


In [194]:
import numpy as np
import spacy
from torch.nn import CosineSimilarity
from nltk.corpus import wordnet
cos = CosineSimilarity(dim=1)
NLP = spacy.load("en_core_web_sm")

tokenizer_args_normal = {
    "return_tensors": "pt",
    "padding": "longest"
}

# 'NOUN', 'PROPN'
# 'VERB'
import re

def find_synonym(token):
    synonym_list = wordnet.synonyms(token)
    for syn in synonym_list:
        if syn:
            return syn[0].replace("_", " ")
    return token

def change_synonym(text, pos_list):
    pos_tokens = NLP(text)
    aux_tok_list = []
    for idx, token in enumerate(pos_tokens):
        if token.pos_ in pos_list:
            token = find_synonym(token.text)
            token = " " + token if idx > 0 else token
            aux_tok_list += [token]
        else:
            token = " " + token.text if token.pos_ != "PUNCT" and idx > 0 else token.text
            aux_tok_list += [token]

    aux_tok_list = "".join(aux_tok_list)
    return aux_tok_list

def get_prediction_model_outputs(premise_hypothesis, model_nli: callable, tokenizer: callable, is_custom=False, **tokenization_args) -> dict:
    premise_hypothesis = change_synonym(premise_hypothesis, ['VERB'])
    tok_output = tokenizer(premise_hypothesis, **tokenization_args)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    tok_output.to(device)
    model_outputs = model_nli(**tok_output)

    probs = torch.softmax(model_outputs.logits, dim=1)
    return probs

get_prediction_model_outputs(data["sent_combined"][0], model_nli=model_nli, tokenizer=tokenizer_nli, **tokenizer_args_normal)

tensor([[0.5965, 0.0623, 0.3411]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [149]:
tokenizer_nli.convert_ids_to_tokens([34857])

['ĠEVs']